First enstablish the SSL via PuTTY, connecting to the tool-login.wmflabs.org, from port 4711 to the destination enwiki.labsdb (at port 3306). Insert your credentials below:

In [1]:
import pymysql
import datetime
import pandas as pd

wikiconn  = pymysql.connect(host='localhost',
                            user='u15068',                 # get from replica.my.cnf
                            password='G5V2xrWbdFTSkyoU',   # get from replica.my.cnf
                            port=4711,
                            db='enwiki_p',
                            charset='utf8')
wikiCur = wikiconn.cursor()

In [6]:
# DEFINE WHAT TO QUARRY IN THE WIKIPEDIA DATABASE:
# for PageLinks: "pl_from", "pagelinks", ["pl_from", "pl_title"]
# for categorylinks: "cl_from","categorylinks",["cl_from","cl_to"]

columnIDs = "pl_from"
table2Q = "pagelinks"
columns2Q = ["pl_from","pl_title"]

# get IDS to Quarry! 
# For EPAs get them from ../WikiSQL/070916_edit_protected_articles.csv
# For NEPAs: ../Sampling/071816_batch1_sampled_ids.csv

for i in range(3,9):
    batchNum = int(i)
    # Get Ids from batch number batchNum
    ids2Q = GetIdsFromCsv(batchNum,"ids_Batch{}.csv".format(batchNum))
    print("Quarry for {} IDS from batch {}, with last ID being: {}".format(len(ids2Q), batchNum, ids2Q[len(ids2Q)-1]))

    # QUARRY DATABASE
    df_link = BatchQueryFromList(ids2Q, columnIDs, table2Q, columns2Q)
    print("Batch {} contains {} {} links".format(batchNum, len(df_link), columns2Q[1]))

    # this DECODES the links
    links_dec = DecodeLinkColumn(df_link, columns2Q[1])

    # save data fo file
    links_dec.to_csv('pl_nepa_Batch{}.csv'.format(batchNum), encoding="utf-8")

Quarry for 9946 IDS from batch 3, with last ID being: 51103467
[13:24:30] fetching chunk 0. Ids from 621 to 320521
[13:24:41] fetching chunk 1. Ids from 321331 to 1266078
[13:24:48] fetching chunk 2. Ids from 1266804 to 3504271
[13:24:54] fetching chunk 3. Ids from 3508376 to 7663767
[13:25:01] fetching chunk 4. Ids from 7667886 to 13991250
[13:25:12] fetching chunk 5. Ids from 14005846 to 20905642
[13:25:18] fetching chunk 6. Ids from 20928073 to 27541495
[13:25:23] fetching chunk 7. Ids from 27557618 to 35353535
[13:25:35] fetching chunk 8. Ids from 35355468 to 42744937
[13:25:40] fetching chunk 9. Ids from 42748567 to 51103467
Batch 3 contains 1737063 pl_title links
Quarry for 9966 IDS from batch 4, with last ID being: 51106337
[13:25:52] fetching chunk 0. Ids from 572 to 304572
[13:26:01] fetching chunk 1. Ids from 304970 to 1181282
[13:26:08] fetching chunk 2. Ids from 1183023 to 3353715
[13:26:15] fetching chunk 3. Ids from 3375153 to 6921685
[13:26:21] fetching chunk 4. Ids from

In [ ]:

batchNum=4
ids2Q = GetIdsFromCsv(batchNum,"ids_Batch{}.csv".format(batchNum))

In [ ]:
idsInChunk = ",".join(map(str,ids2Q))
idsInChunk[:1000]

=================================

The necessary functions follow:

In [2]:
def GetIdsFromCsv(n_batch, path):
    ids = pd.read_csv(path)
    ids2Q = sorted(ids["page_id"])
    return ids2Q

In [3]:
def chunks(l, n):                                          # WATCH OUT! This is not a function, it's a GENERATOR.
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [4]:
def PrintableTimestamp():
    now = datetime.datetime.now().time()
    return "[{:02d}:{:02d}:{:02d}]".format(now.hour,now.minute,now.second)

def BatchQueryFromList(listOfIDs,columnThatContainsIDs,tableToQuery,listOfColumnsToRetrieve):
    '''listOfIDs is a list of keys to retrieve from an SQL table
    columnThatContainsIDs is the column in that table that stores the keys
    tableToQuery is the table containing the information you want
    listOfColumnsToRetrieve is a list containing the SQL DB table column names you want to retrieve
    Returns a pandas dataframe with column names equivalent to those in the DB you queried'''
    baseQ = "SELECT {} FROM {} WHERE {} IN ({})"
    colsForQ = ",".join(listOfColumnsToRetrieve)
    idChunks = chunks(listOfIDs, 1000)
    result_df = pd.DataFrame()
    for ix, chunk in enumerate(idChunks):
        print(PrintableTimestamp() + " fetching chunk {}. Ids from {} to {}".format(ix,chunk[0],chunk[-1]))
        idsInChunk = ",".join(map(str,chunk))
        preppedQ = baseQ.format(colsForQ,tableToQuery,columnThatContainsIDs,idsInChunk)
        wikiCur.execute(preppedQ)
        if not result_df.empty:
            result_df = result_df.append(pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve))
        else:
            result_df = pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve)
    return result_df

In [5]:
# this DECODES the links
def DecodeLinkColumn(dataframe, tableColumn):
    df_link = dataframe
    df_link[tableColumn] = [x.decode("utf-8") for x in dataframe[tableColumn].values]
    return df_link

Some exploratory code

In [ ]:
#for some reason the index of this dataframe has 8 copies
from collections import Counter
print(Counter(link.index)[13])
saved_links = pd.read_csv("pl_nepa_Batch{}.csv".format(batchNum))
#the index is fixed when i save it!!!
print(saved_links[len(saved_links)-3:])
#let's get rid of the strange column 
del saved_links["Unnamed: 0"]
print(saved_links[len(saved_links)-3:])
art_num = set(saved_links["pl_from"])
len(art_num)

In [7]:
saved_links = pd.read_csv("pl_nepa_Batch3.csv")

In [11]:

saved_links[:5]

,pl_from,pl_title
0,621,Actinistia
1,621,Actinopterygii
2,621,Adaptive_radiation
3,621,Adipose_tissue
4,621,Aestivation
